In [1]:
pip install requests pymongo pandas

   ---------------------------------------- 0.0/962.2 kB ? eta -:--:--
   --------------------------------------- 962.2/962.2 kB 13.9 MB/s eta 0:00:00

   ---------------------------------------- 0/2 [dnspython]
   ---------------------------------------- 0/2 [dnspython]
   -------------------- ------------------- 1/2 [pymongo]
   -------------------- ------------------- 1/2 [pymongo]
   -------------------- ------------------- 1/2 [pymongo]
   ---------------------------------------- 2/2 [pymongo]

Note: you may need to restart the kernel to use updated packages.


In [2]:
# pip install requests pymongo

import os
from datetime import datetime, timezone
import requests
from pymongo import MongoClient, ASCENDING
from datetime import datetime, timezone

In [3]:
# =======================
# Config (live-only)
# =======================
API_KEY = "7bdaae7abc6622e671495ef02578a955"   # your real key
CITY    = "London"

MONGO_URI   = os.getenv("MONGO_URI", "mongodb://127.0.0.1:27017")
DB_NAME     = "sales_weatherinfo_db"
WEATHER_COL = "weather"   # existing collection is fine

In [4]:
# =======================
# Helpers
# =======================
def midnight_utc_naive(dt_utc: datetime) -> datetime:
    """Return midnight (00:00:00) in UTC as a naive datetime (Mongo stores as UTC)."""
    dt_utc = dt_utc.astimezone(timezone.utc)
    return datetime(dt_utc.year, dt_utc.month, dt_utc.day, 0, 0, 0, 0)

In [5]:
def ensure_indexes(col):
    # Idempotent; if they already exist, MongoDB keeps them as-is.
    col.create_index([("weather_date", ASCENDING), ("city", ASCENDING)], unique=True)
    col.create_index([("city", ASCENDING), ("weather_date", ASCENDING)])

In [6]:
def upsert_weather(doc: dict, col):
    col.update_one(
        {"weather_date": doc["weather_date"], "city": doc["city"]},
        {"$set": doc},
        upsert=True
    )